# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [4]:
%load_ext dotenv
%dotenv



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [55]:
import os
from glob import glob
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
parquet_files = [x for x in parquet_files if os.path.isfile(x)][0]



In [56]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
CTAS,2008-01-02,33.490002,33.619999,32.259998,32.349998,25.333311,1448400,Industrials,Diversified Support Services,2008
CTAS,2008-01-03,32.610001,33.150002,32.509998,32.970001,25.818834,1334800,Industrials,Diversified Support Services,2008
CTAS,2008-01-04,32.650002,32.740002,31.610001,31.610001,24.753819,1423100,Industrials,Diversified Support Services,2008
CTAS,2008-01-07,31.620001,32.869999,31.620001,32.470001,25.427280,1486500,Industrials,Diversified Support Services,2008
CTAS,2008-01-08,32.540001,32.590000,31.100000,31.100000,24.354439,1557400,Industrials,Diversified Support Services,2008
...,...,...,...,...,...,...,...,...,...,...
CTAS,2008-12-24,21.490000,21.940001,21.350000,21.780001,17.294483,619900,Industrials,Diversified Support Services,2008
CTAS,2008-12-26,21.959999,22.059999,21.690001,21.820000,17.326242,565900,Industrials,Diversified Support Services,2008
CTAS,2008-12-29,21.370001,22.049999,21.370001,21.889999,17.381828,1286500,Industrials,Diversified Support Services,2008


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [64]:
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False
    ).apply(lambda x: x.assign(Close_lag = x['Close'].shift(1))))

dd_feat = (dd_feat.groupby('ticker', group_keys=False
     ).apply(lambda x: x.assign(Adj_Close_lag = x['Adj Close'].shift(1))
     ).assign(returns = lambda x: x['Adj Close']/x['Adj_Close_lag']-1
     ).assign(hi_lo_range = lambda x: x['High']-x['Low']))

/var/folders/9j/x9k9k99508b1c0fbb8n5vj1m0000gp/T/ipykernel_1807/572876171.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False
/var/folders/9j/x9k9k99508b1c0fbb8n5vj1m0000gp/T/ipykernel_1807/572876171.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_feat.groupby('ticker', group_keys=False


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [79]:
df = dd_feat.compute()
df.groupby('ticker', group_keys=False).apply(lambda x: x.assign(roll_avg_ret = x['returns'].rolling(10).mean()))


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,roll_avg_ret
ticker,,,,,,,,,,,,,,,
CTAS,2008-01-02,33.490002,33.619999,32.259998,32.349998,25.333311,1448400,Industrials,Diversified Support Services,2008,NaN,NaN,NaN,1.360001,NaN
CTAS,2008-01-03,32.610001,33.150002,32.509998,32.970001,25.818834,1334800,Industrials,Diversified Support Services,2008,32.349998,25.333311,0.019165,0.640003,NaN
CTAS,2008-01-04,32.650002,32.740002,31.610001,31.610001,24.753819,1423100,Industrials,Diversified Support Services,2008,32.970001,25.818834,-0.041250,1.130001,NaN
CTAS,2008-01-07,31.620001,32.869999,31.620001,32.470001,25.427280,1486500,Industrials,Diversified Support Services,2008,31.610001,24.753819,0.027206,1.249998,NaN
CTAS,2008-01-08,32.540001,32.590000,31.100000,31.100000,24.354439,1557400,Industrials,Diversified Support Services,2008,32.470001,25.427280,-0.042193,1.490000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CTAS,2008-12-24,21.490000,21.940001,21.350000,21.780001,17.294483,619900,Industrials,Diversified Support Services,2008,21.549999,17.111847,0.010673,0.590000,-0.011916
CTAS,2008-12-26,21.959999,22.059999,21.690001,21.820000,17.326242,565900,Industrials,Diversified Support Services,2008,21.780001,17.294483,0.001836,0.369999,-0.008393
CTAS,2008-12-29,21.370001,22.049999,21.370001,21.889999,17.381828,1286500,Industrials,Diversified Support Services,2008,21.820000,17.326242,0.003208,0.679998,-0.011652


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.